<a href="https://colab.research.google.com/github/ChusUEM/RAG-CHAT/blob/main/load_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ElasticDocs GPT Blog
# Loading an embedding from Hugging Face into Elasticsearch

This code will show you how to load a supported embedding model from Hugging Face into an elasticsearch cluster in [Elastic Cloud](https://cloud.elastic.co/)

[Blog - ChatGPT and Elasticsearch: OpenAI meets private data](https://www.elastic.co/blog/chatgpt-elasticsearch-openai-meets-private-data)

# Setup


## Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [1]:
pip -q install eland elasticsearch sentence_transformers transformers torch==1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.6/163.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.


In [2]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

import getpass

/usr/local/lib/python3.10/dist-packages/eland/ml/_optional.py:116: UserWarning: Eland requires version '1.3' or newer of 'sklearn' (version '1.2.2' currently installed). Use pip or conda to update sklearn.
  warnings.warn(msg, UserWarning)
<ipython-input-2-d329d25d548f>:5: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


## Configure elasticsearch authentication.
The recommended authentication approach is using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster level API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [3]:
es_cloud_id = getpass.getpass('RAG-CHAT:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRlMzA3OWFlODYzMDM0ODYzYTUzMWI0NWUzMzc4ZTcwOSQzYWZkMWUwMzRiMGU0MWQyYjI0NDA2M2RkNzY0ZTk5Mw==')
es_user = getpass.getpass('motete-opalos-0e@icloud.com')
es_pass = getpass.getpass('wijsUk-xuptiq-duhjo5')

#es_api_id = getpass.getpass('Enter cluster API key ID:  ')
#es_api_key = getpass.getpass('Enter cluster API key:  ')

KeyboardInterrupt: Interrupted by user

## Connect to Elastic Cloud

In [ ]:
#es = Elasticsearch(cloud_id=es_cloud_id,
#                   api_key=(es_api_id, es_api_key)
#                   )
es = Elasticsearch(cloud_id=es_cloud_id,
                   basic_auth=(es_user, es_pass)
                   )
es.info() # should return cluster info

# Load the model From Hugging Face into Elasticsearch
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page.

When calling `TransformerModel` you specify the HF model id and the task type. You can try specifying `auto` and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific `task_type` values can be viewed in the following code:
[Supported values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

In [ ]:
# Set the model name from Hugging Face and task type
hf_model_id='sentence-transformers/all-distilroberta-v1'
tm = TransformerModel(model_id=hf_model_id, task_type='text_embedding')

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)


# Starting the Model

## View information about the model
This is not required but can be handy to get a model overivew

In [ ]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

## Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [ ]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

## Verify the model started without issue
Should output -> {'routing_state': 'started'}

In [ ]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']